# Analysis Of The Guns In Valorant

### By: Austin Kirwin

## 0. Importing the data and relevant analytical tools

In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score

In [40]:
df_valo = pd.read_csv("valorant-stats.csv")
df_valo

,Name,Weapon Type,Price,Fire Rate,Wall Penetration,Magazine Capacity,HDMG_0,BDMG_0,LDMG_0,HDMG_1,BDMG_1,LDMG_1,HDMG_2,BDMG_2,LDMG_2
0,Classic,Sidearm,0,6.75,Low,12,78,26,22,78,26,22,66,22,18
1,Shorty,Sidearm,200,3.30,Low,2,36,12,10,24,8,6,9,3,2
2,Frenzy,Sidearm,400,10.00,Low,13,78,26,22,63,21,17,63,21,17
3,Ghost,Sidearm,500,6.75,Medium,15,105,33,26,88,25,21,88,25,21
4,Sheriff,Sidearm,800,4.00,High,6,160,55,47,160,55,47,145,50,43
5,Stinger,SMG,1000,18.00,Low,20,67,27,23,62,25,21,62,25,21
6,Spectre,SMG,1600,13.33,Medium,30,78,26,22,66,22,18,66,22,18
7,Bulldog,Rifle,2100,9.15,Medium,24,116,35,30,116,35,30,116,35,30
8,Guardian,Rifle,2500,6.50,Medium,12,195,65,49,195,65,49,195,65,49
9,Phantom,Rifle,2900,11.00,Medium,30,156,39,33,140,35,30,124,31,26


## 1. Introduction

### Primary Research Goal: Determine which gun has the best damage/fire rate based on price for each Weapon Type
### Secondary Research Goal: Build a predictive model to reliably determine what the damage/fire rate would be based on price 

The explanatory variables being considered are "Price", "Fire Rate", "HDMG_*", "BDMG_*", "LDMG_*" where * is a filler for 0, 1, 2. In order to have a consistent amount of damage for each gun (since damage decreases across long distances) the damages will be weighted/averaged before analysis. "Magazine Capacity" and "Wall Penetration" will be ignored, meaning I will be operating under the assumption that there are no objects blocking the bullets that would decrease damage.

Research Motivation: Newer players may want to know which guns have the best damage for the price.

It should be noted that these values could change in future updates or that new guns could be added.

## 2. Dataset Discussion and Manipulation

I found the dataset on Kaggle (https://www.kaggle.com/datasets/aadhavvignesh/valorant-weapon-stats); it is a little outdated which should be considered if variable values are changed in the future.

In [41]:
#Selecting only the relevant columns
df_valo = df_valo[["Name",'Weapon Type','Price','Fire Rate','HDMG_0','BDMG_0','LDMG_0','HDMG_1','BDMG_1','LDMG_1','HDMG_2','BDMG_2','LDMG_2']]
df_valo

,Name,Weapon Type,Price,Fire Rate,HDMG_0,BDMG_0,LDMG_0,HDMG_1,BDMG_1,LDMG_1,HDMG_2,BDMG_2,LDMG_2
0,Classic,Sidearm,0,6.75,78,26,22,78,26,22,66,22,18
1,Shorty,Sidearm,200,3.30,36,12,10,24,8,6,9,3,2
2,Frenzy,Sidearm,400,10.00,78,26,22,63,21,17,63,21,17
3,Ghost,Sidearm,500,6.75,105,33,26,88,25,21,88,25,21
4,Sheriff,Sidearm,800,4.00,160,55,47,160,55,47,145,50,43
5,Stinger,SMG,1000,18.00,67,27,23,62,25,21,62,25,21
6,Spectre,SMG,1600,13.33,78,26,22,66,22,18,66,22,18
7,Bulldog,Rifle,2100,9.15,116,35,30,116,35,30,116,35,30
8,Guardian,Rifle,2500,6.50,195,65,49,195,65,49,195,65,49
9,Phantom,Rifle,2900,11.00,156,39,33,140,35,30,124,31,26


In [42]:
#Creating new column with weighted damage values
df_valo['HDMG_AVG'] = round((df_valo['HDMG_0'] + df_valo['HDMG_1'] + df_valo['HDMG_2']) / 3, 2)
df_valo['BDMG_AVG'] = round((df_valo['BDMG_0'] + df_valo['BDMG_1'] + df_valo['BDMG_2']) / 3, 2)
df_valo['LDMG_AVG'] = round((df_valo['LDMG_0'] + df_valo['LDMG_1'] + df_valo['LDMG_2']) / 3, 2)
df_valo_adj = df_valo[['Name','Weapon Type','Price','Fire Rate','HDMG_AVG','BDMG_AVG','LDMG_AVG']]
df_valo_adj

C:\Users\diamo\AppData\Local\Temp\ipykernel_44644\2589438833.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valo['HDMG_AVG'] = round((df_valo['HDMG_0'] + df_valo['HDMG_1'] + df_valo['HDMG_2']) / 3, 2)


,Name,Weapon Type,Price,Fire Rate,HDMG_AVG,BDMG_AVG,LDMG_AVG
0,Classic,Sidearm,0,6.75,74.00,24.67,20.67
1,Shorty,Sidearm,200,3.30,23.00,7.67,6.00
2,Frenzy,Sidearm,400,10.00,68.00,22.67,18.67
3,Ghost,Sidearm,500,6.75,93.67,27.67,22.67
4,Sheriff,Sidearm,800,4.00,155.00,53.33,45.67
5,Stinger,SMG,1000,18.00,63.67,25.67,21.67
6,Spectre,SMG,1600,13.33,70.00,23.33,19.33
7,Bulldog,Rifle,2100,9.15,116.00,35.00,30.00
8,Guardian,Rifle,2500,6.50,195.00,65.00,49.00
9,Phantom,Rifle,2900,11.00,140.00,35.00,29.67


In [43]:
#Damage will be divided by fire rate for a consistent measurement of dmg/rate
df_valo_adj['DMG/Rate'] = round((df_valo_adj['HDMG_AVG'] + df_valo_adj['BDMG_AVG'] + df_valo_adj['LDMG_AVG']) / (3*df_valo_adj['Fire Rate']), 2)
df_valo_adj

C:\Users\diamo\AppData\Local\Temp\ipykernel_44644\218750101.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valo_adj['DMG/Rate'] = round((df_valo_adj['HDMG_AVG'] + df_valo_adj['BDMG_AVG'] + df_valo_adj['LDMG_AVG']) / (3*df_valo_adj['Fire Rate']), 2)


,Name,Weapon Type,Price,Fire Rate,HDMG_AVG,BDMG_AVG,LDMG_AVG,DMG/Rate
0,Classic,Sidearm,0,6.75,74.00,24.67,20.67,5.89
1,Shorty,Sidearm,200,3.30,23.00,7.67,6.00,3.70
2,Frenzy,Sidearm,400,10.00,68.00,22.67,18.67,3.64
3,Ghost,Sidearm,500,6.75,93.67,27.67,22.67,7.11
4,Sheriff,Sidearm,800,4.00,155.00,53.33,45.67,21.17
5,Stinger,SMG,1000,18.00,63.67,25.67,21.67,2.06
6,Spectre,SMG,1600,13.33,70.00,23.33,19.33,2.82
7,Bulldog,Rifle,2100,9.15,116.00,35.00,30.00,6.59
8,Guardian,Rifle,2500,6.50,195.00,65.00,49.00,15.85
9,Phantom,Rifle,2900,11.00,140.00,35.00,29.67,6.20


## 3. Weapon Type Differences

It is plainly obvious that not all of the guns available have the same "Weapon Type", in order to account for contextual differences in gun usage (shotguns at short range vs snipers far away) weapon types will be analyzed separately.

In [44]:
df_valo_sidearm = df_valo_adj[df_valo_adj["Weapon Type"] == "Sidearm"]
df_valo_sidearm = df_valo_sidearm[['Name','Weapon Type','Price','Fire Rate','DMG/Rate']]
df_valo_sidearm

,Name,Weapon Type,Price,Fire Rate,DMG/Rate
0,Classic,Sidearm,0,6.75,5.89
1,Shorty,Sidearm,200,3.30,3.70
2,Frenzy,Sidearm,400,10.00,3.64
3,Ghost,Sidearm,500,6.75,7.11
4,Sheriff,Sidearm,800,4.00,21.17


In [45]:
df_valo_smg = df_valo_adj[df_valo_adj["Weapon Type"] == "SMG"]
df_valo_smg = df_valo_smg[['Name','Weapon Type','Price','Fire Rate','DMG/Rate']]
df_valo_smg

,Name,Weapon Type,Price,Fire Rate,DMG/Rate
5,Stinger,SMG,1000,18.00,2.06
6,Spectre,SMG,1600,13.33,2.82


In [46]:
df_valo_rifle = df_valo_adj[df_valo_adj["Weapon Type"] == "Rifle"]
df_valo_rifle = df_valo_rifle[['Name','Weapon Type','Price','Fire Rate','DMG/Rate']]
df_valo_rifle

,Name,Weapon Type,Price,Fire Rate,DMG/Rate
7,Bulldog,Rifle,2100,9.15,6.59
8,Guardian,Rifle,2500,6.50,15.85
9,Phantom,Rifle,2900,11.00,6.20
10,Vandal,Rifle,2900,9.25,8.22


In [47]:
df_valo_sniper = df_valo_adj[df_valo_adj["Weapon Type"] == "Sniper"]
df_valo_sniper = df_valo_sniper[['Name','Weapon Type','Price','Fire Rate','DMG/Rate']]
df_valo_sniper

,Name,Weapon Type,Price,Fire Rate,DMG/Rate
11,Marshall,Sniper,1100,1.50,86.22
12,Operator,Sniper,4500,0.75,236.44


In [48]:
df_valo_shotgun = df_valo_adj[df_valo_adj["Weapon Type"] == "Shotgun"]
df_valo_shotgun  = df_valo_shotgun[['Name','Weapon Type','Price','Fire Rate','DMG/Rate']]
df_valo_shotgun

,Name,Weapon Type,Price,Fire Rate,DMG/Rate
13,Bucky,Shotgun,900,1.1,19.80
14,Judge,Shotgun,1500,3.5,4.89


In [49]:
df_valo_heavy = df_valo_adj[df_valo_adj["Weapon Type"] == "Heavy"]
df_valo_heavy = df_valo_heavy[['Name','Weapon Type','Price','Fire Rate','DMG/Rate']]
df_valo_heavy

,Name,Weapon Type,Price,Fire Rate,DMG/Rate
15,Ares,Heavy,1600,10.0,4.13
16,Odin,Heavy,3200,12.0,4.30


## 4. Basic Analysis

In [52]:
# Each 'Weapon Type' will have the 'DMG/Rate' and 'Price' compared to see which has the best ratio.
# 'DMG/Rate' will be considered DPS however this does not necessarily equate the 'DMG/Rate' values to true Damage per Second.

df_valo_sidearm['DPS per Credit'] = df_valo_sidearm['DMG/Rate'] / df_valo_sidearm['Price']

df_valo_smg['DPS per Credit'] = df_valo_smg['DMG/Rate'] / df_valo_smg['Price']

df_valo_rifle['DPS per Credit'] = df_valo_rifle['DMG/Rate'] / df_valo_rifle['Price']

df_valo_sniper['DPS per Credit'] = df_valo_sniper['DMG/Rate'] / df_valo_sniper['Price']

df_valo_shotgun['DPS per Credit'] = df_valo_shotgun['DMG/Rate'] / df_valo_shotgun['Price']

df_valo_heavy['DPS per Credit'] = df_valo_heavy['DMG/Rate'] / df_valo_heavy['Price']

In [53]:
df_valo_sidearm

,Name,Weapon Type,Price,Fire Rate,DMG/Rate,DPS per Credit
0,Classic,Sidearm,0,6.75,5.89,inf
1,Shorty,Sidearm,200,3.30,3.70,0.018500
2,Frenzy,Sidearm,400,10.00,3.64,0.009100
3,Ghost,Sidearm,500,6.75,7.11,0.014220
4,Sheriff,Sidearm,800,4.00,21.17,0.026463


The 'Classic' has inf 'DPS per Credit' on account of it being free; however, the other 'Sidearm' weapons can still be compared.

## 5. Conclusion

In [54]:
df_valo_sidearm

,Name,Weapon Type,Price,Fire Rate,DMG/Rate,DPS per Credit
0,Classic,Sidearm,0,6.75,5.89,inf
1,Shorty,Sidearm,200,3.30,3.70,0.018500
2,Frenzy,Sidearm,400,10.00,3.64,0.009100
3,Ghost,Sidearm,500,6.75,7.11,0.014220
4,Sheriff,Sidearm,800,4.00,21.17,0.026463


Based on the 'DPS per Credit' values, the 'Sheriff' does the most damage for its cost and would be recommended for new players. However, most games try to have some balance between guns, especially secondary weapons, so there might be other aspects of the 'Sheriff' that offset the extra value it has.

In [55]:
df_valo_shotgun

,Name,Weapon Type,Price,Fire Rate,DMG/Rate,DPS per Credit
13,Bucky,Shotgun,900,1.1,19.80,0.02200
14,Judge,Shotgun,1500,3.5,4.89,0.00326


The 'Bucky' has a significantly higher 'DPS per Credit' between the two 'Shotguns' so its value seems drastically better compared to the 'Judge'. However, it should be noted that the damage values in the adjusted data frame show very similar damage values for head, body, and leg shots whereas the 'Judge' has ~3x more 'Fire Rate'.

In [56]:
df_valo_heavy

,Name,Weapon Type,Price,Fire Rate,DMG/Rate,DPS per Credit
15,Ares,Heavy,1600,10.0,4.13,0.002581
16,Odin,Heavy,3200,12.0,4.30,0.001344


The Odin has ~2x 'DPS per Credit' on account of the 'DMG/Rate' of each gun being about the same. In this case, it is very reasonable to assume that there are other differences in between the 'Odin' and the 'Ares' that make up for the similar 'DMG/Rate' between the two weapons.

In [58]:
df_valo_rifle

,Name,Weapon Type,Price,Fire Rate,DMG/Rate,DPS per Credit
7,Bulldog,Rifle,2100,9.15,6.59,0.003138
8,Guardian,Rifle,2500,6.50,15.85,0.006340
9,Phantom,Rifle,2900,11.00,6.20,0.002138
10,Vandal,Rifle,2900,9.25,8.22,0.002834


The 'Guardian' has a slightly better 'DPS per Credit' than the other 'Rifles' which appears to be offset by its much lower 'Fire Rate' comparatively. 

In [59]:
df_valo_smg

,Name,Weapon Type,Price,Fire Rate,DMG/Rate,DPS per Credit
5,Stinger,SMG,1000,18.00,2.06,0.002060
6,Spectre,SMG,1600,13.33,2.82,0.001762


The 'Stinger' and 'Spectre' seem to have nearly the same 'DPS per Credit' so either one could be recommended (again, it should be assumed that other differences exist between the guns that are not listed).

In [60]:
df_valo_sniper

,Name,Weapon Type,Price,Fire Rate,DMG/Rate,DPS per Credit
11,Marshall,Sniper,1100,1.50,86.22,0.078382
12,Operator,Sniper,4500,0.75,236.44,0.052542


Much like the 'SMGs' the 'Snipers' appear to have similar 'DPS per Credit' and should be chosen based on other factors that the data set does not provide.